In [7]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from pandas import Series, DataFrame
import pandas as pd
import re
%pylab inline
import matplotlib.pyplot as plt
from collections import defaultdict

from surprise import SVD
from surprise import Dataset

from surprise import NormalPredictor

from surprise import Reader
from surprise import NMF
from surprise import SVDpp
from surprise import accuracy
from surprise.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


In [8]:
users=pd.read_csv('UserAnimeList.csv')

In [9]:
users

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,1362307973,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,1362923691,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,1367081015,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,1362308037,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,1364399953,NaN
5,karthiga,210,161,0000-00-00,0000-00-00,7,2,NaN,0,1362923826,NaN
6,karthiga,232,70,0000-00-00,0000-00-00,6,2,NaN,0,1362849882,NaN
7,karthiga,233,78,0000-00-00,0000-00-00,6,2,NaN,0,1362893384,NaN
8,karthiga,249,167,0000-00-00,0000-00-00,8,2,NaN,0,1363709086,NaN
9,karthiga,269,366,0000-00-00,0000-00-00,10,2,NaN,0,1362303563,NaN


In [40]:
df =users[['username','anime_id','my_score']].iloc[:2000]

In [61]:
testdata=df[df['my_score']==0]

In [62]:
traindata=df[df['my_score']!=0]

In [41]:
reader = Reader(rating_scale=(1, 2000))

In [63]:
readertrain = Reader(rating_scale=(1, traindata.shape[0]))
readertest = Reader(rating_scale=(1, testdata.shape[0]))

In [67]:
data = Dataset.load_from_df(df[['username','anime_id','my_score']], reader)

In [64]:
train_data=Dataset.load_from_df(traindata[['username','anime_id','my_score']], readertrain)

In [65]:
test_data=Dataset.load_from_df(testdata[['username','anime_id','my_score']], readertest)

In [75]:
trainset, testset = train_test_split(train_data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.5863


1.5863259089309052

In [76]:
algo = SVDpp()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.6057


1.605661354388417

In [77]:
algo = NMF()
trainset, testset = train_test_split(train_data, test_size=.25)
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 2.3284


2.328411875293117

In [85]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
trainset, testset = train_test_split(data, test_size=.25)
#trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
#testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Bas_G [849, 22535, 5667, 2001, 15809, 14075, 433, 16592, 777, 24705]
Damonashu [19, 21603, 226, 177, 934, 486, 4059, 21, 2596, 845]
RedvelvetDaisuki [5958, 1952, 23847, 23283, 8841, 13601, 6211, 18465, 934, 6746]
karthiga [14227, 1735, 857, 853, 10030, 1698, 1914, 233, 232, 4224]
shuzzable [19365, 7148, 4896, 2966, 2581, 2025, 18139, 5355, 13601, 12883]
bskai [21, 66, 30, 401, 2564, 5958, 251, 8769, 2494, 245]
terune_uzumaki [479, 2923, 7082, 6347, 1604]


In [84]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#trainset, testset = train_test_split(train_data, test_size=.25)
trainset = train_data.build_full_trainset()
algo = NMF()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

karthiga [17819, 1535, 268, 13767, 1425, 12355, 6553, 437, 9074, 2904]
RedvelvetDaisuki [21, 801, 60, 5, 966, 189, 5690, 2787, 6, 16668]
Damonashu [16201, 11123, 15439, 578, 5114, 34599, 4814, 17074, 18857, 15117]
bskai [22535, 578, 16668, 19815, 21881, 5341, 801, 12355, 6547, 19]
shuzzable [6553, 12355, 5365, 437, 801, 13767, 19, 5941, 1425, 17819]
terune_uzumaki [12355, 801, 6553, 210, 1575, 19, 22535, 2904, 1425, 17074]
Bas_G [801, 12355, 210, 17074, 1, 31043, 22, 34599, 90, 685]


In [86]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#trainset, testset = train_test_split(train_data, test_size=.25)
trainset = train_data.build_full_trainset()
algo = SVDpp()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

karthiga [1535, 4181, 1425, 1008, 5114, 19815, 431, 245, 2132, 6547]
RedvelvetDaisuki [21, 5, 443, 28171, 6547, 4181, 2970, 22, 1015, 5118]
Damonashu [17074, 9926, 34599, 3092, 6553, 6707, 170, 7785, 981, 578]
bskai [1425, 437, 19, 12355, 34599, 877, 7193, 431, 7647, 6707]
shuzzable [12355, 1535, 5, 23283, 4181, 7054, 22535, 34599, 801, 97]
terune_uzumaki [21, 1535, 1, 12355, 7054, 5114, 22535, 5, 23283, 19]
Bas_G [437, 6553, 34599, 12355, 21, 13767, 17819, 5365, 19, 3092]


In [89]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):


    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

trainset, testset = train_test_split(test_data, test_size=.99)
# First train an SVD algorithm on the movielens dataset.
#trainset, testset = train_test_split(train_data, test_size=.25)
trainset = train_data.build_full_trainset()
algo = SVD()
algo.fit(trainset)


# Than predict ratings for all pairs (u, i) that are NOT in the training set.
#testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

shuzzable [8525, 5118, 4382, 10721, 5681, 6746, 10020, 14075, 72, 16524]
RedvelvetDaisuki [24439, 10863, 834, 8937, 5958, 1690, 2593, 11617, 12413, 8426]
Bas_G [8142, 6707, 25157, 3731, 18857, 10163, 18679, 1311, 3712, 23273]
Damonashu [486, 170, 10162, 6594, 276, 5114, 14075, 3588, 18153, 66]
bskai [7647, 7858, 8246, 877, 766, 2967, 6773, 355, 5277, 1067]
